# YOLO Person Detection Model Training

This notebook trains a YOLO model for person detection and position tracking for crowd analysis.

## Dataset Requirements
- Images with people in various crowd scenarios
- YOLO format annotations (class_id x_center y_center width height)
- Class 0: person

## Output
- Trained YOLO model weights
- Position data for each detected person
- Bounding box coordinates normalized to image dimensions


In [ ]:
# Install required packages
%pip install ultralytics torch torchvision opencv-python numpy matplotlib pillow

# Import libraries
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
from ultralytics import YOLO
import os
import time
from google.colab import drive


In [ ]:
# Mount Google Drive to access dataset
drive.mount('/content/drive')

# Set dataset path (update this path to your dataset location)
DATASET_PATH = '/content/drive/MyDrive/CrowdProject/dataset'

# Verify dataset structure
if os.path.exists(DATASET_PATH):
    print(f"Dataset found at: {DATASET_PATH}")
    print("\nDataset structure:")
    for root, dirs, files in os.walk(DATASET_PATH):
        level = root.replace(DATASET_PATH, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Show first 5 files
            print(f"{subindent}{file}")
        if len(files) > 5:
            print(f"{subindent}... and {len(files) - 5} more files")
else:
    print(f"Dataset not found at: {DATASET_PATH}")
    print("Please upload your dataset to Google Drive and update DATASET_PATH")


In [ ]:
# Create YOLO dataset configuration file
config_content = f"""
# YOLO Dataset Configuration for Person Detection
path: {DATASET_PATH}
train: images/train
val: images/val
test: images/test

# Classes
nc: 1  # number of classes
names: ['person']  # class names
"""

# Write config file
config_path = '/content/dataset.yaml'
with open(config_path, 'w') as f:
    f.write(config_content)

print(f"Dataset configuration created at: {config_path}")
print("\nConfiguration content:")
print(config_content)


In [ ]:
# Load YOLO model (using pre-trained YOLOv8)
model = YOLO('yolov8n.pt')  # nano version for faster training

# Check if CUDA is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


In [ ]:
# Train the model
results = model.train(
    data=config_path,
    epochs=100,
    imgsz=640,
    batch=16,
    device=device,
    project='/content/runs/detect',
    name='person_detection',
    save_period=10,  # Save checkpoint every 10 epochs
    patience=20,     # Early stopping patience
    lr0=0.01,        # Initial learning rate
    lrf=0.01,        # Final learning rate
    momentum=0.937,  # SGD momentum
    weight_decay=0.0005,  # Weight decay
    warmup_epochs=3,      # Warmup epochs
    warmup_momentum=0.8,  # Warmup momentum
    warmup_bias_lr=0.1,   # Warmup bias learning rate
    box=7.5,              # Box loss gain
    cls=0.5,              # Class loss gain
    dfl=1.5,              # DFL loss gain
    augment=True,         # Enable augmentation
    hsv_h=0.015,          # Image HSV-Hue augmentation
    hsv_s=0.7,            # Image HSV-Saturation augmentation
    hsv_v=0.4,            # Image HSV-Value augmentation
    degrees=0.0,          # Image rotation (+/- deg)
    translate=0.1,        # Image translation (+/- fraction)
    scale=0.5,            # Image scale (+/- gain)
    shear=0.0,            # Image shear (+/- deg)
    perspective=0.0,      # Image perspective (+/- fraction)
    flipud=0.0,           # Image flip up-down (probability)
    fliplr=0.5,           # Image flip left-right (probability)
    mosaic=1.0,           # Image mosaic (probability)
    mixup=0.0,            # Image mixup (probability)
    copy_paste=0.0,       # Segment copy-paste (probability)
)

print("Training completed!")
